# DFTpy on a Jupyter Notebook
This is meant to be executed on a notebook on the Google Colaboratory

## Install DFTpy and download a pseudopotential file

In [ ]:
!pip install git+https://gitlab.com/pavanello-research-group/dftpy.git@dev
!wget https://raw.githubusercontent.com/EACcodes/local-pseudopotentials/master/BLPS/LDA/reci/al.lda.recpot

## import some modules

In [ ]:
from dftpy.ions import Ions
from dftpy.field import DirectField
from dftpy.grid import DirectGrid
from dftpy.functional import LocalPseudo, Functional, TotalFunctional
from dftpy.formats import io
from dftpy.math_utils import ecut2nr
from dftpy.time_data import TimeData
from dftpy.optimization import Optimization
from dftpy.mpi import sprint

## pseudopotential file

In [ ]:

file1='al.lda.recpot'
PP_list = {'Al': file1}

## build the ions or read from file

In [ ]:
from ase.build import bulk
atoms = bulk('Al', 'fcc', a=4.05, cubic=True)
ions = Ions.from_ase(atoms)

## make a grid

In [ ]:
nr = ecut2nr(ecut=35, lattice=ions.cell)
grid = DirectGrid(lattice=ions.cell, nr=nr)
sprint('The final grid size is ', nr)

##  build local pseudo, and generate guess density

In [ ]:
PSEUDO = LocalPseudo(grid = grid, ions=ions, PP_list=PP_list)

rho_ini = DirectField(grid=grid)
rho_ini[:] = ions.get_ncharges()/ions.cell.volume

## instance KEDF, XC and HARTREE functionals

In [ ]:
KE = Functional(type='KEDF',name='TFvW')
XC = Functional(type='XC',name='LDA')
HARTREE = Functional(type='HARTREE')

## instance DFTpy evaluator

In [ ]:
evaluator = TotalFunctional(KE=KE, XC=XC, HARTREE=HARTREE, PSEUDO=PSEUDO)

## instance and execute DFTpy density optimizer

In [ ]:
optimization_options = {'econv' : 1e-6*ions.nat}
opt = Optimization(EnergyEvaluator=evaluator, optimization_options = optimization_options,
        optimization_method = 'TN')

rho = opt.optimize_rho(guess_rho=rho_ini)

## evaluate final energy

In [ ]:
energy = evaluator.Energy(rho=rho, ions=ions)
print('Energy (a.u.)', energy)

##  print the timing

In [ ]:
TimeData.output(lprint=True, sort='cost')

## Visualize with scikit-image and matplotlib

In [ ]:
!pip install scikit-image matplotlib

In [ ]:
from dftpy.visualize import view

In [ ]:
%matplotlib inline
view(data=rho)